In [ ]:
import pandas as pd
import numpy as np
import os

# Traemos el dataframe que utilizaremos para entrenar nuestro modelo
df_train = pd.read_csv('../partitions/train.csv', sep=';')
df_test = pd.read_csv('../partitions/test.csv', sep=';')

matriz_datos_train = feature_extraction(df_train)
matriz_datos_test = feature_extraction(df_test)

if not os.path.exists('../features'):
    os.mkdir('../features')

np.save('../features/matriz_datos_train.npy', matriz_datos_train)
np.save('../features/matriz_datos_test.npy', matriz_datos_test)

In [ ]:
import cv2
import matplotlib.pyplot as plt


def feature_extraction(df):

    fingerprint = []

    for i in range(0, len(df)):
        file = df.ID[i]
        img = cv2.imread('../Material/Images/' + file)
        rnfl_mask = cv2.imread('../Material/RNFL_masks/' + file, 0)
        retina_mask = cv2.imread('../Material/Retina_masks/' + file, 0)

        # Visualizacion
        fig, ax = plt.subplots(1, 3)
        ax[0].imshow(img, cmap='gray'), ax[0].set_title('Imagen')
        ax[1].imshow(rnfl_mask, cmap='gray'), ax[1].set_title('RNFL')
        ax[2].imshow(retina_mask, cmap='gray'), ax[2].set_title('Retina')

        # A partir de la imagen obtenida, vamos a extraer las características
        # para poder determinar si la muestra se asocia con un paciente sano
        # o con un paciente con glaucoma

        # Para poder representar las características de una imagen haremos uso de las mascaras
        # que ya han sido obtenidas de manera manual o automática

        # Muestras con un RNFL grueso se asocian con un paciente sano
        # Por otro lado, muestras con un RNFL delgado se asocian con un paciente con glaucoma
        # Ya que al ser la capa muy delgada, indica que las células están desgastadas

        # ESTADÍSTICOS UNIDIMENSIONALES en la RNFL
        # Calcularemos el grosor en cada columna de la imagen para así saber el ancho de la muestra RNFL
        thickness_rnfl = []
        for j in range(0, rnfl_mask.shape[1]):
            # Encontramos la posición en la columna donde los pixeles son blancos
            pos = np.where(rnfl_mask[:,j] == 255)

            # Restamos la posicón del último pixel blanco menos la posición del primer pixel blanco
            thickness_rnfl.append(pos[0][-1] - pos[0][0])

        thickness_rnfl = np.array(thickness_rnfl)

        # Características basadas en medidas de tendencia central para no utilizar los 768 valores
        media = np.mean(thickness_rnfl)
        mediana = np.median(thickness_rnfl)

        # Características basadas en medidas de dispersión
        devest = np.std(thickness_rnfl)

        # Características de distribución
        from scipy import stats

        asimetria = stats.skew(thickness_rnfl)
        curtosis = stats.kurtosis(thickness_rnfl)

        # Otras características como el valor mínimo y el maximo
        minimo = np.min(thickness_rnfl)
        maximo = np.max(thickness_rnfl)

        # Hemos pasado de 768 a 7 características con estadísticos unidimensionales
        # Fingerprint de la RNFL
        features_RNFL = [media, mediana, devest, asimetria, curtosis, minimo, maximo]


        # CARACTERÍSTICAS BI DIMENSIONALES en la estructura de la retina
        # Queremos centrarnos solamente en nuestra área de interes
        # así que haremos la bounding box para a partir de ahí extraer las características
        from skimage.measure import regionprops

        prop = regionprops(retina_mask)
        bb = prop[0].bbox

        # Ahora haremos un crop de la máscara de la retina para obtener solo el área de interes
        retina = img[bb[0]:bb[2], bb[1]:bb[3], 0]

        # Una vez que tenemos el crop, ahora vamos a extraer características de textura

        # Gray Level Cocurrence Matrix (GLCM)
        from skimage.feature import greycomatrix, greycoprops

        # Definmos un angulo de 90 por que nos interesa el cambio de intensidad vertical
        GLCM = greycomatrix(retina, distances=[2], angles=[90], , levels=256, symetric=True, normed=True)

        contraste = greycoprops(GLCM, 'contrast')[0, 0]
        disimilitud = greycoprops(GLCM, 'disimilarity')[0, 0]
        homogeneidad = greycoprops(GLCM, 'homogeneity')[0, 0]
        ASM = greycoprops(GLCM, 'ASM')[0, 0]
        energia = greycoprops(GLCM, 'energy')[0, 0]
        correlacion = greycoprops(GLCM, 'correlation')[0, 0]

        # Local Binary Patterns (LBP)
        from skimage.feature import local_binary_pattern

        R=1 # radui
        P=8*R # numero de vecinos
        lbp_image = local_binary_pattern(retina, P, method='uniform')

        # Extraemos el histograma para usarlo como descriptor de textura
        lbp_image = np.uint8(lbp_image)

        hist_lbp = cv2.calcHist([lbp_image.ravel()], [0], None, [P+2], [0, P+2])

        # Normalizamos el histograma
        hist_lbp = hist_lbp.astype('float')

        hist_lbp /= (hist_lbp.sum() + 1e-7)

        hist_lbp = hist_lbp.tolist()
        hist_lbp = [item for sublist in hist_lbp for item in sublist]

        # Visualización de la imagen LBP y el histograma
        plt.imshow(lbp_image, cmap='gray')
        plt.show()

        plt.plot(hist_lbp)
        plt.grid(True)
        plt.show()

        # Características de textura (Fingerprint de la retina)
        features_retina = [contraste, disimilitud, homogeneidad, ASM, energia, correlacion] + hist_lbp

        # Extraer la información de la clase
        # Para etiquetar una variable categorica
        if df.Class[i] == 'Healthy':
            etiqueta = [0]
        else:
            etiqueta = [1]

        fingerprint.append(features_RNFL + features_retina + etiqueta)

    matrizdatos = np.array(fingerprint)
    
    return matrizdatos
